In [1]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

Using TensorFlow backend.


In [5]:
df = pd.read_csv('train.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 26: invalid start byte

In [4]:
print(data.shape)

(49972, 2)


In [5]:
model_lstm = Sequential()
model_lstm.add(Embedding(20000, 100, input_length=50))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model_lstm.fit(data, np.array(labels), validation_split=0.4, epochs=3)

ValueError: Error when checking input: expected embedding_1_input to have shape (50,) but got array with shape (2,)

In [11]:

def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [12]:
model_conv = create_conv_model()
model_conv.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

NameError: name 'vocabulary_size' is not defined

In [ ]:
df_save = pd.DataFrame(data)
df_label = pd.DataFrame(np.array(labels))


In [ ]:
result = pd.concat([df_save, df_label], axis = 1)

In [ ]:
result.to_csv('train_dense_word_vectors.csv', index=False)

In [ ]:
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_glove.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

In [ ]:
lstm_embds = model_lstm.layers[0].get_weights()[0]

In [ ]:

conv_embds = model_conv.layers[0].get_weights()[0]

In [ ]:
glove_emds = model_glove.layers[0].get_weights()[0]

In [ ]:
word_list = []
for word, i in tokenizer.word_index.items():
    word_list.append(word)

In [ ]:
def plot_words(data, start, stop, step):
    trace = go.Scatter(
        x = data[start:stop:step,0], 
        y = data[start:stop:step, 1],
        mode = 'markers',
        text= word_list[start:stop:step]
    )
    layout = dict(title= 't-SNE 1 vs t-SNE 2',
                  yaxis = dict(title='t-SNE 2'),
                  xaxis = dict(title='t-SNE 1'),
                  hovermode= 'closest')
    fig = dict(data = [trace], layout= layout)
    py.iplot(fig)

In [ ]:
lstm_tsne_embds = TSNE(n_components=2).fit_transform(lstm_embds)

In [ ]:
plot_words(lstm_tsne_embds, 0, 2000, 1)

In [ ]:
conv_tsne_embds = TSNE(n_components=2).fit_transform(conv_embds)

In [ ]:
plot_words(conv_tsne_embds, 0, 2000, 1)

In [ ]:

glove_tsne_embds = TSNE(n_components=2).fit_transform(glove_emds)

In [ ]:
plot_words(glove_tsne_embds, 0, 2000, 1)

In [ ]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

In [ ]:

df['tokenized'] = df.apply(lambda row : nltk.word_tokenize(row['text']), axis=1)

In [ ]:

model_w2v = Word2Vec(df['tokenized'], size=100)

In [ ]:
X = model_w2v[model_w2v.wv.vocab]

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
tsvd = TruncatedSVD(n_components=5, n_iter=10)
result = tsvd.fit_transform(X)

In [ ]:
result.shape

In [ ]:
tsvd_word_list = []
words = list(model_w2v.wv.vocab)
for i, word in enumerate(words):
    tsvd_word_list.append(word)

trace = go.Scatter(
    x = result[0:number_of_words, 0], 
    y = result[0:number_of_words, 1],
    mode = 'markers',
    text= tsvd_word_list[0:number_of_words]
)

layout = dict(title= 'SVD 1 vs SVD 2',
              yaxis = dict(title='SVD 2'),
              xaxis = dict(title='SVD 1'),
              hovermode= 'closest')

fig = dict(data = [trace], layout= layout)
py.iplot(fig)